In [7]:
from querymodule import fume_query, query_to_list, total_energy, total_time_sash_open
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import tz
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

points = pd.read_csv("pointnames.csv")

In [8]:
def has_been_closed(sash_point,server,start,end):
    query = fume_query(sash_point,server,start,end)
    inches = [i[0] for i in query]
    ratio = inches.count(min(inches))/len(inches)
    return(ratio)

sash_point = points["Sash"][0]
server  = points["Server"][0]
time_start = str(datetime(2022, 12,1))
time_end = str(datetime(2022, 12,31))


sash_points = points["Sash"]
ratio_list = [has_been_closed(sash_points[i], server,time_start,time_end) for i in range(len(sash_points))]
print(ratio_list)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200